# Notebook 08 – LLM-as-a-Judge y Control de Calidad

Objetivo:
Implementar un sistema de evaluación automática de las respuestas generadas
para reducir alucinaciones y mejorar la calidad del contenido.


¿Qué es LLM-as-a-Judge?

Usar un segundo LLM para:

Evaluar la salida de otro LLM

Aplicar criterios objetivos

Tomar decisiones automáticas

💡 Ejemplos reales:

Chatbots empresariales

Copilots

Sistemas científicos

Moderación de contenido

🎯 CRITERIOS QUE VAMOS A USAR

1️⃣ Claridad
2️⃣ Corrección factual
3️⃣ Utilidad para el usuario
4️⃣ Adecuación al contexto
5️⃣ Riesgo de alucinación

PROMPT DEL JUEZ (TEORÍA)

Un buen juez:

NO reescribe

NO añade contenido

SOLO evalúa

El output debe ser:

estructurado

fácil de parsear

usable por código

📁backend/app/models/agents/judge_agent.py

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from app.models.llms.groq_llm import get_groq_llm

llm = get_groq_llm()

judge_prompt = ChatPromptTemplate.from_template("""
Eres un evaluador experto de contenido generado por IA.

Evalúa la siguiente respuesta según estos criterios:
- Claridad
- Corrección factual
- Utilidad
- Adecuación al contexto
- Riesgo de alucinación

Devuelve SOLO este formato:

SCORE: <número de 0 a 10>
FEEDBACK: <breve explicación>

Pregunta original:
{question}

Respuesta generada:
{answer}
""")

def evaluate_answer(question: str, answer: str) -> dict:
    response = llm.invoke(
        judge_prompt.format(
            question=question,
            answer=answer
        )
    )

    text = response.content

    score = 0
    feedback = ""

    for line in text.splitlines():
        if line.startswith("SCORE"):
            score = int(line.split(":")[1].strip())
        if line.startswith("FEEDBACK"):
            feedback = line.split(":")[1].strip()

    return {
        "score": score,
        "feedback": feedback
    }


INTEGRAR EL JUEZ EN EL SISTEMA

Ahora vamos a usar el juez DESPUÉS de generar contenido.

📁backend/app/services/content_service.py

🔁 MODIFICA la función generate_content

In [ ]:
from app.models.agents.judge_agent import evaluate_answer

📌 Añade este import arriba

🔁 REEMPLAZA la función generate_content por esta versión:

In [ ]:
def generate_content(question: str) -> dict:
    memory.add("user", question)

    strategy = route(question)

    if strategy == "graph_rag":
        answer = graph_rag_answer(
            text=question,
            question=question
        )
    elif strategy == "multiagent":
        answer = marketing_agent(question)
    else:
        answer = rag_answer(question)

    evaluation = evaluate_answer(question, answer)

    memory.add("assistant", answer)

    return {
        "answer": answer,
        "evaluation": evaluation,
        "strategy_used": strategy
    }
    

QUÉ HEMOS CONSEGUIDO

Ahora tu sistema:

✅ Genera contenido
✅ Decide estrategia
✅ Evalúa su propia calidad
✅ Devuelve métricas
✅ Reduce alucinaciones

Esto ya es:

Arquitectura de IA responsable

In [ ]:
{
  "answer": "La inteligencia artificial es...",
  "evaluation": {
    "score": 8,
    "feedback": "Respuesta clara y bien estructurada, aunque podría profundizar más en ejemplos."
  },
  "strategy_used": "rag"
}
